<a href="https://colab.research.google.com/github/ntamminga1/DeepLearningChess/blob/main/Train_chess_cheating_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-chess
import chess.pgn
from IPython.display import clear_output
import math
from typing import NamedTuple, Tuple
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import gc
import pickle
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
#Neural network largely inspired from AlphaZero

def initialize_weights(net: nn.Module) -> None:

    for module in net.modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            nn.init.kaiming_uniform_(module.weight, nonlinearity='relu')

            if module.bias is not None:
                nn.init.zeros_(module.bias)


class ResNetBlock(nn.Module):
    #Basic redisual block

    def __init__(
        self,
        num_filters: int,
    ) -> None:
        super().__init__()

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(
                in_channels=num_filters,
                out_channels=num_filters,
                kernel_size=3,
                stride=1,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(num_features=num_filters),
            nn.ReLU(),
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(
                in_channels=num_filters,
                out_channels=num_filters,
                kernel_size=3,
                stride=1,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(num_features=num_filters),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        residual = x
        out = self.conv_block1(x)
        out = self.conv_block2(out)
        out += residual
        out = F.relu(out)
        return out


class AlphaZeroNet(nn.Module):

    def __init__(
        self,
        input_shape: Tuple,
        num_res_block: int = 20,
        num_filters: int = 32,
        num_fc_units: int = 32,
    ) -> None:
        super().__init__()
        c, h, w = input_shape

        num_padding =1

        conv_out = h * w

        # First convolutional block
        self.conv_block = nn.Sequential(
            nn.Conv2d(
                in_channels=c,
                out_channels=num_filters,
                kernel_size=3,
                stride=1,
                padding=num_padding,
                bias=False,
            ),
            nn.BatchNorm2d(num_features=num_filters),
            nn.ReLU(),
        )

        # Residual blocks
        self.res_blocks = nn.Sequential(*[ResNetBlock(num_filters) for _ in range(num_res_block)])


        self.value = nn.Sequential(
            nn.Conv2d(
                in_channels=num_filters,
                out_channels=1,
                kernel_size=1,
                stride=1,
                bias=False,
            ),
            nn.BatchNorm2d(num_features=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(1 * conv_out, num_fc_units),
            nn.ReLU(),
            nn.Linear(num_fc_units, 1)
        )

        initialize_weights(self)

    def forward(self, x: torch.Tensor) -> torch.tensor:

        conv_block_out = self.conv_block(x)
        features = self.res_blocks(conv_block_out)

        # Predict whether the move is human or engine move
        value = self.value(features)

        return value

In [ ]:
channel_dict = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }

#Create a tensor form a FEN position and move
def fen_to_data(fen, moves):
    input = torch.zeros((20,8,8))
    pieces, turn, castling, enpassant = fen.split(" ")
    board = pieces.split('/')
    for rank in range(8):
        file = 0
        for piece in board[rank]:
            if piece.isnumeric():
                file += int(piece)
            else:
                input[channel_dict[piece], rank, file] = 1
                file += 1
    input[12] = 1*(turn == 'w')
    input[13] = 1*('K' in castling)
    input[14] = 1*('Q' in castling)
    input[15] = 1*('k' in castling)
    input[16] = 1*('q' in castling)
    if enpassant != '-':
        input[17, int(enpassant[1])-1, ord(enpassant[0])-97] = 1

    inputs = []
    labels = []
    for i, move_set in enumerate([moves[0], moves[2]]):
        for move in move_set:
            current_input = input.clone()

            current_input[18, int(move[1])-1, ord(move[0])-97] = 1
            current_input[19, int(move[3])-1, ord(move[2])-97] = 1

            inputs.append(current_input)
            labels.append(1-i)

    return inputs, labels


In [ ]:
#Load in data
data_dict = {}
for i in tqdm(range(14)):
    with open(f'/content/drive/MyDrive/chess_data/final_dict{i:03d}.pkl', 'rb') as f:
        data_dict = data_dict | pickle.load(f)

100%|██████████| 14/14 [01:45<00:00,  7.51s/it]


In [ ]:
#Seperate the data into a train and a test set

#fraction = 0.9
#train_inds = np.random.choice(len(data_dict), int(len(data_dict)*0.9), replace = False)
#with open(f'/content/drive/MyDrive/chess_data/train_inds.pkl', 'wb') as f:
#    pickle.dump(train_inds, f)

In [ ]:
#Seperate the data into a train and a test set

with open(f'/content/drive/MyDrive/chess_data/train_inds.pkl', 'rb') as f:
    train_inds = pickle.load(f)
test_inds = sorted(list(set(range(len(data_dict)))-set(train_inds)))
fens = list(data_dict.keys())
train_fens = [fens[i] for i in train_inds]
test_fens = [fens[i] for i in test_inds]

In [ ]:
#Take a subset of the train data to hopefully speed up training

#train_subset_inds = np.random.choice(len(train_fens), 1000000, replace = False)
#with open(f'/content/drive/MyDrive/chess_data/train_subset_inds.pkl', 'wb') as f:
#    pickle.dump(train_subset_inds, f)

In [ ]:
#Convert train data to tensors and move them to GPU

train_inputs = []
train_labels = []
with open(f'/content/drive/MyDrive/chess_data/train_subset_inds.pkl', 'rb') as f:
    train_subset_inds = pickle.load(f)
for i in tqdm(train_subset_inds):
    inputs, labels = fen_to_data(train_fens[i], data_dict[train_fens[i]])
    train_inputs += inputs
    train_labels += labels
train_inputs = torch.stack(train_inputs)
train_labels = torch.tensor(train_labels, dtype = torch.float32)
gpu_train_inputs = train_inputs.to(device = device)
gpu_train_labels = train_labels.to(device = device)

100%|██████████| 1000000/1000000 [05:36<00:00, 2969.52it/s]


In [ ]:
#Convert test data to tensors and move them to GPU

test_inputs = []
test_labels = []
for fen in tqdm(test_fens):
    inputs, labels = fen_to_data(fen, data_dict[fen])
    test_inputs += inputs
    test_labels += labels
test_inputs = torch.stack(test_inputs)
test_labels = torch.tensor(test_labels, dtype = torch.float32)
gpu_test_inputs = test_inputs.to(device = device)
gpu_test_labels = test_labels.to(device = device)

100%|██████████| 531382/531382 [02:43<00:00, 3255.85it/s]


In [ ]:
#Initialize neural network

neural_net = AlphaZeroNet((20,8,8), 10, 32, 32).to(device = device)
files = [int(filename[10:14]) for filename in os.listdir('/content/drive/MyDrive/chess_data/small_model') if filename.startswith("neural_net")]
if len(files)>0:
    last_i = max(files)
    neural_net.load_state_dict(torch.load(f'/content/drive/MyDrive/chess_data/small_model/neural_net{last_i:04d}.pt'))
else:
    last_i = 0
    torch.save(neural_net.state_dict(), f'/content/drive/MyDrive/chess_data/small_model/neural_net{0:04d}.pt')
optim = torch.optim.SGD(neural_net.parameters(), lr=1e-2, momentum=0.9, weight_decay = 10**-5)
loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
#Train neural network and test accuracy

train_loss = 0
train_count = 0
n = 64
inds = np.arange(len(gpu_train_inputs))
with tqdm() as pbar:
    for i in range(last_i+1, 100000):
        np.random.shuffle(inds)
        neural_net.train()
        for j in range(len(inds)//n):

            batch_inputs = gpu_train_inputs[inds[j*n:(j+1)*n]]
            batch_labels = gpu_train_labels[inds[j*n:(j+1)*n]]

            optim.zero_grad()

            outputs = neural_net(batch_inputs).squeeze(dim = 1)
            loss = loss_fn(outputs,batch_labels)
            loss.backward()
            train_loss += loss.detach()
            train_count += 1
            optim.step()


            pbar.update(1)

        test_inds = np.random.choice(len(gpu_test_inputs), n, replace = False)
        batch_inputs = gpu_test_inputs[test_inds]
        batch_labels = gpu_test_labels[test_inds]

        neural_net.eval()
        with torch.no_grad():
            outputs = neural_net(batch_inputs).squeeze(dim = 1)
            test_loss = loss_fn(outputs,batch_labels)

        torch.save(neural_net.state_dict(), f'/content/drive/MyDrive/chess_data/small_model/neural_net{i:04d}.pt')

        print(f"   Train Loss = {train_loss.to(device = 'cpu').numpy()/train_count}   Test Loss = {test_loss.to(device = 'cpu').numpy()}")
        train_loss = 0
        train_count = 0

27211it [04:58, 78.88it/s]

   Train Loss = 0.634682180215457   Test Loss = 0.7056843042373657


54413it [09:52, 76.58it/s]

   Train Loss = 0.6233760582120009   Test Loss = 0.5903697609901428


81603it [14:45, 74.33it/s]

...

5629997it [15:59:49, 82.29it/s] 

   Train Loss = 0.5465988853730054   Test Loss = 0.6704790592193604


5657200it [16:04:28, 84.96it/s]

   Train Loss = 0.5465318494811199   Test Loss = 0.5722806453704834


5663033it [16:05:28, 101.01it/s]